Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Install module

In [ ]:
!pip install geopandas imageio numpy  pandas rasterio scipy \
             shapely svgwrite tensorflow-gpu==2.10.0 tflearn rtree

In [ ]:
%cd "/content/drive/My Drive/road_detection/data"

NICFI download (download_nicfi_target.py)(Not required if you use data that has already been prepared)

In [ ]:
#usege
#!python3 "/content/drive/My Drive/road_detection/prepare_dataset/download_nicfi_target.py"\
# [target_date] [start date] [start_ID] [API_key]  path/to/json_file

!python3 "/content/drive/My Drive/road_detection/prepare_dataset/download_nicfi_target.py" \
 2021-08 202108 106688 [API_key] "/content/drive/My Drive/road_detection/prepare_dataset/config/Brazil_test_106688.json"

merge geojson (merge.py) (Not required if you use data that has already been prepared)

In [ ]:
%cd "/content/drive/My Drive/road_detection/data/polygon_data/"
#merge json data
import geopandas as gpd
import pandas as pd
import glob
import os

outputname = "merge.json"

folder =os.getcwd()

gdf_merge = gpd.GeoDataFrame()
files = glob.glob(os.path.join(folder ,'*.json'))
for f in files:
    gdf_file1 = gpd.read_file(f)
    gdf_merge = gpd.GeoDataFrame(pd.concat([gdf_merge ,gdf_file1]))

gdf_merge.to_file(driver='GeoJSON', filename=os.path.join(folder,outputname))

Set L4 GPU

Road detection(test_Brazil_target_area.py)

In [ ]:
%cd "/content/drive/My Drive/road_detection/data/"
!python3 "/content/drive/My Drive/road_detection/model/test_Brazil_target_area.py" \
-model_save ./tmp -instance_id test -image_size 352 \
-model_recover "/content/drive/My Drive/road_detection/pretrained_model/model300000" \
-mode test -dataset "/content/drive/My Drive/road_detection/data/nicfi_dataset/" \
-mesh_polygon \
"/content/drive/My Drive/road_detection/data/polygon_data/202104-202108/merge_202104-202108.json"
#"/content/drive/My Drive/road_detection/data/polygon_data/merge.json"

Set CPU without High-RAM


Mount Google Drive again

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Install module again

In [ ]:
!pip install geopandas numpy pandas rasterio

Create directories and move detecting road files

In [ ]:
import os
import shutil
###Setting arbitrary name ###
target_name = "temp"

## main ###
%cd "/content/drive/My Drive/road_detection/data"
folder =os.getcwd()

path1= "outputs_"+ target_name
path2= "outputs_"+ target_name + "_merge"
path3= "outputs_"+ target_name + "_merge_utm"
path4= "outputs_"+target_name + "_merge_utm_value_over30/"
path5= "outputs_"+target_name+ "_merge_utm_value_over30_removal_fault_roads"
path6 = "outputs_"+target_name+ "_merge_utm_value_over30_removal_fault_roads_eliminated/"
path7 ="outputs_"+target_name + "_merge_utm_value_over30_cumulation_roads/"
path8 ="outputs_"+target_name+ "_merge_utm_value_over30_cumulation_roads_error_removed/"
path9 ="outputs_"+target_name + "_proximity_value/"
path10 ="outputs_mesh_centroid_point_road_distance_value"

#os.makedirs(os.path.join(folder ,path1), exist_ok=True)
os.makedirs(os.path.join(folder ,path2), exist_ok=True)
os.makedirs(os.path.join(folder ,path3), exist_ok=True)
os.makedirs(os.path.join(folder ,path4), exist_ok=True)
os.makedirs(os.path.join(folder ,path5), exist_ok=True)
os.makedirs(os.path.join(folder ,path6), exist_ok=True)
os.makedirs(os.path.join(folder ,path7), exist_ok=True)
os.makedirs(os.path.join(folder ,path8), exist_ok=True)
os.makedirs(os.path.join(folder ,path9), exist_ok=True)
os.makedirs(os.path.join(folder ,path10), exist_ok=True)

shutil.move('outputs', path1)


Mosaicking and Converting Geodetic System(rasterio_merge_warp.py)

if average longitude is less than 78 degree and more than 72 degree, then dst_crs = "EPSG:32718" (WGS 84 / UTM zone 18S).<BR>
if average longitude is less than 72 degree and more than 66 degree, then dst_crs = "EPSG:32719" (WGS 84 / UTM zone 19S).<BR>
if average longitude is less than 66 degree and more than 60 degree, then dst_crs = "EPSG:32720" (WGS 84 / UTM zone 20S).<BR>
if average longitude is less than 60 degree and more than 54 degree, then dst_crs = "EPSG:32721" (WGS 84 / UTM zone 21S).<BR>
if average longitude is less than 54 degree and more than 48 degree, then dst_crs = "EPSG:32722" (WGS 84 / UTM zone 22S).<BR>
if average longitude is less than 48 degree and more than 42 degree, then dst_crs = "EPSG:32723" (WGS 84 / UTM zone 23S).<BR>
if average longitude is less than 42 degree and more than 36 degree, then dst_crs = "EPSG:32724" (WGS 84 / UTM zone 24S).<BR>
if average longitude is less than 36 degree and more than 30 degree, then dst_crs = "EPSG:32725" (WGS 84 / UTM zone 25S).

In [ ]:
%cd "/content/drive/My Drive/road_detection/data"
import rasterio
import glob
import sys
from rasterio.merge import merge
from rasterio.warp import reproject, Resampling, calculate_default_transform

#input target list
#target_list =['Porto_Velho_10000','Humaita_20000','Altamira_30000','Vista_Alegre_do_Abuna_40000','Novo_Progresso_50000','Sao_Felix_do_Xingu_60000','S6W57_70000','S7W57_80000']
target_list =['temp']
#data = ['201606','201612','201706','201712','201806','201812','201906','201912','202006','202009','202010','202011','202012','202101','202102','202103','202104','202105','202106','202107','202108','202109','202110','202111','202112','202201','202202','202203','202204','202205','202206','202207','202208','202209']
data = ['202104','202105','202106','202107','202108']

for target in target_list:
  inputpath ="./outputs_"+target + "/"
  outputpath1="./outputs_"+target + "_merge/"
  outputpath2="./outputs_"+target + "_merge_utm/"

  if target == 'Vista_Alegre_do_Abuna_40000':
    dst_crs = 'EPSG:32719'
  elif target == 'Porto_Velho_10000' or target == 'Humaita_20000':
    dst_crs = 'EPSG:32720'
  elif target == 'Novo_Progresso_50000' or  \
       target == 'S6W57_70000' or target == 'S7W57_80000' or target == 'temp':
    dst_crs = 'EPSG:32721'
  elif target == 'Porto_Velho_10000' or target == 'Altamira_30000' or target == 'Sao_Felix_do_Xingu_60000':
    dst_crs = 'EPSG:32722'
  else:
    print(target + " is projection error!")
    sys.exit()

  for i in data:
    sat_path = glob.glob(inputpath + i +'*_sat.tif')
    key_path = glob.glob(inputpath + i + '*_output_keypoints.tif')

    if len(sat_path) == 0:
      print("Error!")

    elif len(sat_path) > 1:
      sat_dest, sat_output = merge(sat_path)

      with rasterio.open(sat_path[0]) as src:
          out_meta = src.meta.copy()
      out_meta.update({"driver": "GTiff",
                   "height": sat_dest.shape[1],
                   "width": sat_dest.shape[2],
                   "transform": sat_output})
      out_meta.update(compress='lzw')

      with rasterio.open(outputpath1 + i + "merge_sat.tif", "w", **out_meta) as sat_out:
          sat_out.write(sat_dest)

      key_dest, key_output = merge(key_path)

      with rasterio.open(key_path[0]) as src2:
          out_meta2 = src2.meta.copy()
      out_meta2.update({"driver": "GTiff",
                   "height": key_dest.shape[1],
                   "width": key_dest.shape[2],
                   "transform": key_output})
      out_meta2.update(compress='lzw')

      with rasterio.open(outputpath1 + i + "merge_keypoints.tif", "w", **out_meta2) as key_out:
          key_out.write(key_dest)

          transform, width, height = calculate_default_transform(
          key_out.crs, dst_crs, key_out.width, key_out.height, *key_out.bounds)
          kwargs = out_meta2.copy()
          kwargs.update({
              'crs': dst_crs,
              'transform': transform,
              'width': width,
              'height': height
          })
          kwargs.update(dtype=rasterio.int8,compress='lzw')

          with rasterio.open(outputpath2+ i + "merge_keypoints_"+ target + "_utm.tif" , 'w', **kwargs) as dst:
            for j in range(1, key_out.count + 1):
                reproject(
                    source=rasterio.band(key_out, j),
                    destination=rasterio.band(dst, j),
                    src_transform=key_out.transform,
                    src_crs=key_out.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest)


    else:
      with rasterio.open(key_path[0]) as key_out:
           transform, width, height = calculate_default_transform(
              key_out.crs, dst_crs, key_out.width, key_out.height, *key_out.bounds)
           kwargs = key_out.meta.copy()
           kwargs.update({
               'crs': dst_crs,
              'transform': transform,
               'width': width,
               'height': height
           })
           kwargs.update(compress='lzw')

           with rasterio.open(outputpath2+ i + "merge_keypoints_"+ target + "_utm.tif" , 'w', **kwargs) as dst:
            for j in range(1, key_out.count + 1):
                reproject(
                    source=rasterio.band(key_out, j),
                    destination=rasterio.band(dst, j),
                    src_transform=key_out.transform,
                    src_crs=key_out.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest)

    print(i + " is finished!")


Binarization of Road Data(rasterio_calc_over30.py)

In [ ]:
import os
import numpy as np
import rasterio
import glob
import sys

threshold = 30

#target_list =['Porto_Velho_10000','Humaita_20000','Altamira_30000','Vista_Alegre_do_Abuna_40000','Novo_Progresso_50000','Sao_Felix_do_Xingu_60000','S6W57_70000','S7W57_80000']
target_list =['temp']

for target in target_list:
    inputpath = "./outputs_"+target + "_merge_utm/"
    outputpath = "./outputs_"+target+ "_merge_utm_value_over30/"

    data_path = glob.glob(inputpath + '*.tif')

    for filepath in data_path:
        basename = os.path.basename(filepath)
        date = basename[:6]
        output_tiff = outputpath + date + "merge_keypoints_"+ target + "_value_utm_over30.tif"
        img1 = rasterio.open(filepath)
        input_meta = img1.meta
        input_meta.update(dtype=rasterio.int8,compress='lzw')

        input_img = img1.read(1)

        output_img = np.where(input_img <= threshold, 0 ,1)

        with rasterio.open(output_tiff,'w', **input_meta) as outdata:
            outdata.write_band(1,output_img.astype(rasterio.int8))

        print(date ," finished!")


#Post Processing#

if use in 202105, 202106, 202107 and 202108, then<BR>
data = ['202105','202106','202107','202108']<BR>

(Not use '202104' due to cloud)

Remove falsely detected pixels(rasterio_removal_fault_roads.py)

In [ ]:
import rasterio
import numpy as np
import shutil

target = 'temp'

#data = ['201606','201612','201706','201712','201806','201812','201906','201912','202006','202009','202010','202011','202012','202101','202102','202103','202104','202105','202106','202107','202108','202109','202110','202111','202112','202201','202202','202203','202204','202205','202206','202207','202208','202209']
data = ['202105','202106','202107','202108']

inputpath ="./outputs_"+target+ "_merge_utm_value_over30/"
outputpath ="./outputs_"+target+ "_merge_utm_value_over30_removal_fault_roads/"

filename ="merge_keypoints_"+target+ "_value_utm_over30.tif"


j = 0
for j in range(len(data)-2):
    print(data[j])
    print(data[j+1])
    print(data[j+2])
    print("")

    input_tiff = inputpath + data[j] + filename
    input_tiff1 = inputpath + data[j+1] + filename
    input_tiff2 = inputpath + data[j+2] + filename

    output_tiff = outputpath + data[j] + "merge_keypoints_" + target + "_over30_removal_fault_roads.tif"
    img = rasterio.open(input_tiff)
    input_meta = img.meta
    input_img = img.read(1)

    img1 = rasterio.open(input_tiff1)
    input1_meta = img1.meta
    input1_img = img1.read(1)

    img2 = rasterio.open(input_tiff2)
    input2_meta = img2.meta
    input2_img = img2.read(1)

    calc1 = np.where(input1_img+input2_img>=1,1,0)
    result = input_img * calc1

    input_meta.update(dtype=rasterio.int8,compress='lzw')

    with rasterio.open(output_tiff,'w', **input_meta) as outdata:
        outdata.write_band(1,result.astype(rasterio.int8))

    print(data[j] ," finished!")

output_tiff1 = outputpath + data[j+1] + "merge_keypoints_" + target + "_over30_removal_fault_roads.tif"
output_tiff2 = outputpath + data[j+2] + "merge_keypoints_" + target + "_over30_removal_fault_roads.tif"

shutil.copy2(input_tiff1,output_tiff1)
shutil.copy2(input_tiff2,output_tiff2)

Remove small falsely detected road(rasterio_clump.py)

In [7]:
import rasterio
import numpy as np
import pandas as pd
from scipy.ndimage import label, generate_binary_structure

target ='temp'

#data = ['201606','201612','201706','201712','201806','201812','201906','201912','202006','202009','202010','202011','202012','202101','202102','202103','202104','202105','202106','202107','202108','202109','202110','202111','202112','202201','202202','202203','202204','202205','202206','202207','202208','202209']
data = ['202105','202106','202107','202108']

th = 150

#inputpath ="./outputs_Porto_Velho_10000_over30_removal_fault_roads/"
#outputpath ="./outputs_Porto_Velho_10000_over30_removal_fault_roads_eliminated/"

inputpath ="./outputs_"+target+ "_merge_utm_value_over30_removal_fault_roads/"
outputpath ="./outputs_"+target+ "_merge_utm_value_over30_removal_fault_roads_eliminated/"

s = generate_binary_structure(2,2)

for i in data:
    input_tiff = inputpath +i+ "merge_keypoints_" + target + "_over30_removal_fault_roads.tif"
    output_tiff = outputpath + i + "merge_keypoints_" + target + "_over30_removal_fault_roads_eliminated.tif"

    with rasterio.open(input_tiff) as img1:
        img = img1.read(1)
        img_meta = img1.meta
        img_meta.update(
        dtype=rasterio.uint8,
        #count=1,
        compress='lzw')

    labeled_img, num_features = label(img, structure=s)
    labeled_img2 = np.where(img==0,0,labeled_img)
    np_count = np.array(np.unique(labeled_img2, return_counts=True)).T

    col = ['id','count']
    df_count = pd.DataFrame(np_count)
    df_count2 = df_count.set_axis(['id','count'],axis=1).drop(df_count.index[0])
    df_count_valid = df_count2[df_count2['count'] >= th]
    df_count_valid_id = df_count_valid['id']

    np_count_valid_id = df_count_valid_id.values
    lines = np_count_valid_id.shape[0]
    for i in range(lines):
        labeled_img2  = np.where(labeled_img2  == np_count_valid_id[i],-1,labeled_img2)

    img_out = np.where(labeled_img2 >= 0,0,1)
    with rasterio.open(output_tiff, "w", **img_meta) as dataset2:
        dataset2.write(img_out,1)


Creation of cumulative roads(rasterio_road_cumulation.py)

In [ ]:
import numpy as np
import os
import shutil
import rasterio

target ='temp'

#data = ['201606','201612','201706','201712','201806','201812','201906','201912','202006','202009','202010','202011','202012','202101','202102','202103','202104','202105','202106','202107','202108','202109','202110','202111','202112','202201','202202','202203','202204','202205','202206','202207','202208','202209']
data = ['202105','202106','202107','202108']

inputpath ="./outputs_"+target+ "_merge_utm_value_over30_removal_fault_roads_eliminated/"
outputpath ="./outputs_"+target+ "_merge_utm_value_over30_cumulation_roads/"

inputfilename =  "merge_keypoints_" + target + "_over30_removal_fault_roads_eliminated.tif"
outputfilename = "merge_keypoints_" + target + "_over30_road_cumulation.tif"

j = 0
for j in range(len(data)-1):
    print(data[j+1])
    print("")

    input_tiff1 = inputpath + data[j+1] + inputfilename
    output_tiff = outputpath + data[j+1] + outputfilename

    if j == 0:

       input_tiff = inputpath + data[j] + inputfilename
       #copy first file
       output1 = outputpath + data[j] + outputfilename
       shutil.copy2(input_tiff,output1)

       img = rasterio.open(input_tiff)
       input_meta = img.meta
       input_meta.update(dtype=rasterio.int8,compress='lzw')
       input_img = img.read(1)

    else:
       input_tiff = outputpath + data[j] + outputfilename
       img = rasterio.open(input_tiff)
       input_img = img.read(1)

    img1 = rasterio.open(input_tiff1)
    input1_img = img1.read(1)

    result = np.where(input_img+input1_img>=1,1,0)

    with rasterio.open(output_tiff,'w', **input_meta) as outdata:
        outdata.write_band(1,result.astype(rasterio.int8))

    print(data[j] ," finished!")
    input_img = np.zeros(input_img.shape)
    input_img = np.copy(input1_img)

(optional) Remove falsely detected road manually (rasterio_road_error_removal.py)

In [ ]:
import numpy as np
import geopandas as gpd
import rasterio
from rasterio import features

target ='temp'

#Create polygon
remove_area_path = "./temp_error.json"
#remove_area_path = "./filename_error.json"


#data = ['201606','201612','201706','201712','201806','201812','201906','201912','202006','202009','202010','202011','202012','202101','202102','202103','202104','202105','202106','202107','202108','202109','202110','202111','202112','202201','202202','202203','202204','202205','202206','202207','202208','202209']
data = ['202105','202106','202107','202108']

inputpath ="./outputs_"+target+ "_merge_utm_value_over30_cumulation_roads/"
outputpath ="./outputs_"+target+ "_merge_utm_value_over30_cumulation_roads_error_removed/"

remove_area = gpd.read_file(remove_area_path)

# Get list of geometries for all features in vector file
geom = [shapes for shapes in remove_area.geometry]

for i in data:
    input_tiff = inputpath +i+ "merge_keypoints_" + target + "_over30_road_cumulation.tif"
    output_tiff = outputpath + i + "merge_keypoints_" + target + "_over30_road_cumulation_error_removed.tif"

    img1 = rasterio.open(input_tiff)
    input_meta = img1.meta
    input_meta.update(dtype=rasterio.int8,compress='lzw')
    input_img = img1.read(1)

    remove_area_rasterized = features.rasterize(geom,
                                out_shape = img1.shape,
                                #fill = 0,
                                out = None,
                                transform = img1.transform,
                                all_touched = False,
                                default_value = 1,
                                dtype = None)

    output = np.where(remove_area_rasterized == 1,0,input_img)

    with rasterio.open(output_tiff, "w", **input_meta) as dataset:
        dataset.write(output,indexes = 1)

Calculation of the shortest distance from the centroid of each 1km mesh to the road（rasterio_compute_proximity.py）

In [ ]:
import os

import rasterio
from scipy import ndimage

target ="temp"

#data = ['201606','201612','201706','201712','201806','201812','201906','201912','202006','202009','202010','202011','202012','202101','202102','202103','202104','202105','202106','202107','202108','202109','202110','202111','202112','202201','202202','202203','202204','202205','202206','202207','202208','202209']
data = ['202105','202106','202107','202108']

#inputfilename = "merge_keypoints_" + target + "_over30_road_cumulation_error_removed.tif"
inputfilename =  "merge_keypoints_" + target + "_over30_road_cumulation.tif"

outputfilename = "merge_keypoints_" + target + "_proximity_value_over30.tif"

inputpath ="./outputs_"+target+ "_merge_utm_value_over30_cumulation_roads/"
outputpath ="./outputs_"+target+ "_proximity_value/"

cellsize = 4.777314267160000405

for i in data:
    input_tiff = inputpath + i + inputfilename
    output_tiff = outputpath + i + outputfilename

    img = rasterio.open(input_tiff)
    input_meta = img.meta

    input_meta.update(dtype=rasterio.uint32,compress='lzw')


    input_img = img.read(1)
    output = (ndimage.distance_transform_edt(input_img!=1) * cellsize).astype('int32')

    with rasterio.open(output_tiff,'w', **input_meta) as outdata:
        outdata.write_band(1,output.astype(rasterio.int32))

    print(i ," finished!")

create mesh polygon (if not created)(A part of create_dataset.py)

In [ ]:
%cd "/content/drive/My Drive/road_detection/data"
from shapely.geometry import Polygon
import geopandas as gpd
import pandas as pd
import numpy as np
import sys
import math
import os
import glob


import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

##########
# config #
##########
#input ID
#"Porto Velho" is 1, "Humaita" is 2, "Altamira" is 3, "Vista Alegre do Abuna" is 4, "Novo Progresso" is 5,
#"Sao_Felix_do_Xingu" is 6, "S6W57" is 7,and "S7W57" is 8. If adding other area, please put the number in order from 9.
ID = 11

#Input geojson file name
geojson_file = "merge.json"

#set json file path
#files = "/path/to/geojson"
folder =os.getcwd()
file = os.path.join(folder ,'polygon_data',geojson_file)
polygon_folder_dir = os.path.join(folder, 'polygon')

#mesh size(m)(default:1000)
resolution = 1000


##########
#  main  #
##########
gdf_file = gpd.read_file(file)

#calculate minx,miny, maxx and maxy
gdf_file_bounds = gdf_file.bounds
west = gdf_file_bounds.loc[0, "minx"]
south = gdf_file_bounds.loc[0, "miny"]
east = gdf_file_bounds.loc[0, "maxx"]
north = gdf_file_bounds.loc[0, "maxy"]

# if west >= east or south >= north, then exit.
if  west >= east:
    print("ERROR! East must be greater than west.")
    sys.exit()
if south >= north:
    print("ERROR! North must be greater than south.")
    sys.exit()

#create target polygon area
gdf_polygon_area = gpd.GeoDataFrame({'target': ['1'], 'geometry':[Polygon([(west, south), (west, north),(east, north), (east, south)])]},crs="EPSG:4326")

average = (west + east) /2
#define Universal Transverse Mercator(UTM) coordinate system using (west + east)/2
if  average >= -78 and average <= -72:
    #UTM 18S
    crs = "EPSG:32718"
elif average >= -72 and average <= -66:
    #UTM 19S
    crs = "EPSG:32719"
elif average >= -66 and average <= -60:
    #UTM 20S
    crs = "EPSG:32720"
elif average >= -60 and average <= -54:
    #UTM 21S
    crs = "EPSG:32721"
elif average >= -54 and average <= -48:
    #UTM 22S
    crs = "EPSG:32722"
elif average >= -48 and average <= -42:
      #UTM 23S
    crs = "EPSG:32723"
elif average >= -42 and average <= -36:
      #UTM 24S
    crs = "EPSG:32724"
elif average >= -36 and average <= -30:
      #UTM 25sS
    crs = "EPSG:32725"
else:
    print("ERROR! out of range or cross the boundary")
    sys.exit()

#set UTM coordinate system
gdf_polygon_area_UTM = gdf_polygon_area.to_crs(crs)

#calculate minx,miny, maxx and maxy
gdf_polygon_area_UTM_bounds = gdf_polygon_area_UTM.bounds
minx = math.ceil(gdf_polygon_area_UTM_bounds.loc[0, "minx"] / resolution) * resolution
miny = math.ceil(gdf_polygon_area_UTM_bounds.loc[0, "miny"] / resolution) * resolution
maxx = math.floor(gdf_polygon_area_UTM_bounds.loc[0, "maxx"] / resolution) * resolution
maxy = math.floor(gdf_polygon_area_UTM_bounds.loc[0, "maxy"] / resolution) * resolution

#create the list of the cols and rows
cols = list(np.arange(minx, maxx + resolution, resolution))
rows = list(np.arange(miny, maxy + resolution, resolution))

#create empty geodatabase
gdf_grid_polygon = gpd.GeoDataFrame()

#create grid polygon
for x in cols[:-1]:
    for y in rows[:-1]:
        cell = gpd.GeoDataFrame([Polygon([(x,y), (x + resolution, y), (x + resolution, y + resolution), (x, y + resolution)])])
        cell["left"] = x
        cell["top"] = y + resolution
        cell["right"] = x + resolution
        cell["bottom"] = y
        gdf_grid_polygon = gpd.GeoDataFrame(pd.concat([gdf_grid_polygon,cell], ignore_index=True))

gdf_grid_polygon = gdf_grid_polygon.set_geometry(0)
gdf_grid_polygon = gdf_grid_polygon.set_crs(crs)

gdf_grid_polygon["centroid_x"] = gdf_grid_polygon[0].centroid.x
gdf_grid_polygon["centroid_y"] = gdf_grid_polygon[0].centroid.y

#extract grid polygon within the target area
gdf_grid_polygon_target_area = gdf_grid_polygon.sjoin(gdf_polygon_area_UTM, how="inner", predicate="within").drop(['index_right', 'target'], axis=1)

#set ID
gdf_grid_polygon_target_area['ID'] = range(ID*1000000 + 1, ID*1000000 + len(gdf_grid_polygon_target_area.index) + 1)

#create directory
os.makedirs(os.path.join(polygon_folder_dir, str(ID)), exist_ok=True)

#export grid polygon geojson file to main directry
gdf_grid_polygon_target_area.to_file(driver='GeoJSON', filename=os.path.join(polygon_folder_dir, str(ID), "1km_mesh_with_centroid.geojson"))


Calculate the shortest distance from the centroids of each 1 km mesh to the road(rasterio_extract_point_value.py)

In [ ]:
%cd "/content/drive/My Drive/road_detection/data"
import geopandas as gpd
import pandas as pd
import rasterio
import warnings
warnings.simplefilter('ignore')

##########
# config #
##########
#input ID
#"Porto Velho" is 1, "Humaita" is 2, "Altamira" is 3, "Vista Alegre do Abuna" is 4, "Novo Progresso" is 5,
#"Sao_Felix_do_Xingu" is 6, "S6W57" is 7,and "S7W57" is 8. If adding other area, please put the number in order from 9.
ID = 10

target ="temp"

#data = ['201606','201612','201706','201712','201806','201812','201906','201912','202006','202009','202011','202101','202104','202105','202106','202107','202108','202109','202110','202111','202205','202206','202207','202208','202209']
data = ['202105','202106','202107','202108']

##########
#  main  #
##########

folder =os.getcwd()
polygon_folder_dir = os.path.join(folder, 'polygon')

inputpath ="outputs_"+target+ "_proximity_value/"
gdf_mesh =  gpd.read_file(os.path.join(polygon_folder_dir, str(ID), "1km_mesh_with_centroid.geojson"))

gdf_points = gdf_mesh[['ID', 'centroid_x', 'centroid_y']]

output_csv =os.path.join("outputs_mesh_centroid_point_road_distance_value", target + "_centroid_point_value.csv")

df_points_merge = pd.DataFrame(gdf_points).drop(columns=['centroid_x', 'centroid_y'])

coord_list = [(x, y) for x, y in zip(gdf_points["centroid_x"], gdf_points["centroid_y"])]

for i in data:
    input_tiff = inputpath + i +  "merge_keypoints_" + target + "_proximity_value_over30.tif"

    src = rasterio.open(input_tiff)

    gdf_points['value1'] = [x for x in src.sample(coord_list)]
    df_points = pd.DataFrame(gdf_points).filter(items=['value1'])
    df_points_merge[i] = df_points['value1'].astype(int)

    print(i ," finished!")

df_points_merge.to_csv(output_csv,index=False)


Creation of monthly data(road_Brazil_per_month.py)

In [ ]:
import pandas as pd
import os
target_area_list = ["temp"]

startyear = 2021
startmonth = 5
endyear = 2021
endmonth = 8

#startyear = 2016
#startmonth = 6
#endyear = 2022
#endmonth = 9

main_dir =os.path.join(os.getcwd(), "outputs_mesh_centroid_point_road_distance_value")
#main_dir = "/path/to/"

for target in target_area_list:

    #import csv
    df_data =pd.read_csv(os.path.join(main_dir, target + "_centroid_point_value.csv"))

    df_result = df_data[["ID"]]


    flag0 = 0
    year_month_name = 0


    for i in range(endyear - startyear + 1):
        year = startyear + i
        print(year)
        for month in range(1,13):

            if year == startyear and month < startmonth:
                    pass

            #after 2022-10-08, break
            elif year == endyear and month == endmonth + 1:
                break

            else:

                date1 = str(year) + str(month).zfill(2)
                if date1 in df_data:
                    df_date_data = df_data[[date1]]
                    year_month_name = date1

                else:
                    df_date_data = df_date_data.rename(columns={year_month_name: date1})
                    year_month_name = date1

                df_result = pd.concat([df_result,df_date_data], axis=1)

    df_result.to_csv(os.path.join(main_dir,"road_1km_mesh_"+target +".csv"), index=False)


Change to within 5km or not and in 1km unit(road_5km_threshold_1km_unit.py)

In [ ]:
import pandas as pd
import math
import os

target_area_list = ["temp"]

main_dir =os.path.join(os.getcwd(), "outputs_mesh_centroid_point_road_distance_value")
#main_dir = "/path/to/"

for target in target_area_list:

    road_path = os.path.join(main_dir, "road_1km_mesh_"+target +".csv")
    export_path_5km = os.path.join(main_dir, "road_5km_"+target +".csv")
    export_path_1km_unit = os.path.join(main_dir, "road_1km_unit_"+target +".csv")

    df_road =  pd.read_csv(road_path,index_col=0, header=0)
    data = df_road.columns.values

    df_road_1km_unit = df_road.copy()

    for i in data:
        print(i)
        df_road[i] = df_road[i].apply(lambda x : 1 if x <= 5000 else 0)
        df_road_1km_unit[i] = df_road_1km_unit[i].apply(lambda x : math.ceil(x/1000))

    df_road.to_csv(export_path_5km, header=True, index=True)

    df_road_1km_unit.to_csv(export_path_1km_unit, header=True, index=True)